In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np
import src.models
from exp_ntk import run

In [3]:
def get_str_l(l, precision=2):
    l_str = []
    for el in l:
        l_str.append(f"{el:.{precision}f}")
    return l_str
    

In [4]:
seed = 0

data_params = dict(
    learning_setting = "inductive", # or "transdructive"
    classes = 2,
    n_trn_labeled = 600,
    n_trn_unlabeled = 0,
    n_val = 200,
    n_test = 200,
    sigma = 1,
    avg_within_class_degree = 1.58 * 2,
    avg_between_class_degree = 0.37 * 2,
    K = 1.5,
    seed = 0 # used to generate the dataset & data split
)

model_params = dict(
    label = "GCN",
    model = "GCN",
    normalization = "row_normalization",
    depth = 1,
    regularizer = 1e-8
)

attack_params = dict(
    attack = "random",
    #epsilon_list = [0, 0.01, 0.025, 0.05, 0.10, 0.25, 0.50, 1, 2.5, 5, 10],
    epsilon_list = [0],
    attack_setting = "evasion" # or "poisioning"
)

verbosity_params = dict(
    debug_lvl = "warning"
)  

other_params = dict(
    device = "0",
    dtype = torch.float64,
    allow_tf32 = False
)

In [7]:
def run_exp(n_seeds, data_params, model_params, attack_params, verbosity_params, other_params):
    acc_l = []
    min_ypred = []
    max_ypred = []
    cond = []
    min_M = []
    max_M = []
    min_ntklabeled = []
    max_ntklabeled = []
    min_ntkunlabeled = []
    max_ntkunlabeled = []
    for seed in range(n_seeds):
        data_params["seed"] = seed
        res = run(data_params, model_params, attack_params, verbosity_params, other_params, seed)
        acc_l.append(res["accuracy_l"][0])
        min_ypred.append(res["min_ypred"][0])
        max_ypred.append(res["max_ypred"][0])
        min_M.append(res["min_M"][0])
        max_M.append(res["max_M"][0])
        min_ntklabeled.append(res["min_ntklabeled"][0])
        max_ntklabeled.append(res["max_ntklabeled"][0])
        min_ntkunlabeled.append(res["min_ntkunlabeled"][0])
        max_ntkunlabeled.append(res["max_ntkunlabeled"][0])
        cond.append(res["cond"])
    print(f"Accuracy: {get_str_l(acc_l)}")
    print(f"Min y_pred: {get_str_l(min_ypred)}")
    print(f"Max y_pred: {get_str_l(max_ypred)}")
    print(f"Min M: {get_str_l(min_M)}")
    print(f"Max M: {get_str_l(max_M)}")
    print(f"Min NTK_labeled: {get_str_l(min_ntklabeled)}")
    print(f"Max NTK_labeled: {get_str_l(max_ntklabeled)}")
    print(f"Min NTK_unlabeled: {get_str_l(min_ntkunlabeled)}")
    print(f"Max NTK_unlabeled: {get_str_l(max_ntkunlabeled)}")
    print(f"Condition: {get_str_l(cond, precision=0)}")

In [44]:
data_params["learning_setting"] = "transductive"
data_params["seed"] = 0
run(data_params, model_params, attack_params, verbosity_params, other_params, seed)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
        1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
        1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
        0, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
0.8849999904632568


{'accuracy_l': [0.8849999904632568],
 'epsilon_l': [0],
 'min_ypred': [-1.6868791415714233],
 'max_ypred': [2.0859966172742785],
 'min_M': [1.0],
 'max_M': [1.0],
 'min_ntklabeled': [-0.7442963601856576],
 'max_ntklabeled': [72.55201349661894],
 'min_ntkunlabeled': [-1.9895214628109565],
 'max_ntkunlabeled': [19.413256536604973],
 'cond': 924.0456698483973}

## Experiments with using LU Factorization as solver

In [30]:
data_params["learning_setting"] = "inductive"
data_params["n_trn_labeled"] = 400
model_params["regularizer"] = 1
model_params["solver"] = "LU"
n_seeds = 10
run_exp(n_seeds, data_params, model_params, attack_params, verbosity_params, other_params)

Accuracy: ['0.88', '0.88', '0.84', '0.86', '0.87', '0.88', '0.88', '0.87', '0.88', '0.90']
Min y_pred: ['-21.17', '-35.34', '-35.85', '-29.83', '-23.16', '-30.05', '-19.88', '-20.58', '-30.13', '-22.48']
Max y_pred: ['23.62', '41.48', '16.84', '47.61', '19.82', '30.46', '40.67', '17.82', '21.06', '22.07']
Min M: ['1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00']
Max M: ['1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00']
Min NTK_labeled: ['-2.54', '-2.58', '-2.22', '-2.26', '-1.88', '-3.52', '-1.37', '-2.13', '-1.89', '-1.94']
Max NTK_labeled: ['72.55', '63.80', '64.94', '61.22', '80.16', '71.08', '45.96', '56.18', '62.07', '69.64']
Min NTK_unlabeled: ['-1.99', '-1.24', '-1.25', '-1.03', '-0.93', '-1.45', '-1.57', '-1.44', '-2.17', '-1.18']
Max NTK_unlabeled: ['19.41', '23.67', '20.12', '12.79', '22.19', '24.25', '16.31', '17.83', '21.95', '21.79']
Condition: ['1192', '1165', '1245', '1202', '1191', '1190', '1115', '1204', '1235

In [46]:
data_params["learning_setting"] = "inductive"
data_params["n_trn_labeled"] = 400
model_params["regularizer"] = 1
model_params["solver"] = "QR"
n_seeds = 10
run_exp(n_seeds, data_params, model_params, attack_params, verbosity_params, other_params)

Accuracy: ['0.88', '0.88', '0.84', '0.86', '0.87', '0.88', '0.88', '0.87', '0.88', '0.90']
Min y_pred: ['-21.17', '-35.34', '-35.85', '-29.83', '-23.16', '-30.05', '-19.88', '-20.58', '-30.13', '-22.48']
Max y_pred: ['23.62', '41.48', '16.84', '47.61', '19.82', '30.46', '40.67', '17.82', '21.06', '22.07']
Min M: ['1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00']
Max M: ['1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00']
Min NTK_labeled: ['-2.54', '-2.58', '-2.22', '-2.26', '-1.88', '-3.52', '-1.37', '-2.13', '-1.89', '-1.94']
Max NTK_labeled: ['72.55', '63.80', '64.94', '61.22', '80.16', '71.08', '45.96', '56.18', '62.07', '69.64']
Min NTK_unlabeled: ['-1.99', '-1.24', '-1.25', '-1.03', '-0.93', '-1.45', '-1.57', '-1.44', '-2.17', '-1.18']
Max NTK_unlabeled: ['19.41', '23.67', '20.12', '12.79', '22.19', '24.25', '16.31', '17.83', '21.95', '21.79']
Condition: ['1192', '1165', '1245', '1202', '1191', '1190', '1115', '1204', '1235

## Experiments with new conditioner

In [11]:
data_params["learning_setting"] = "inductive"
data_params["n_trn_labeled"] = 400
model_params["regularizer"] = 1e-8
n_seeds = 10
run_exp(n_seeds, data_params, model_params, attack_params, verbosity_params, other_params)

Accuracy: ['0.73', '0.63', '0.56', '0.55', '0.69', '0.73', '0.65', '0.45', '0.83', '0.50']
Min y_pred: ['-179772.62', '-117061431.84', '-39621.95', '-3022641.79', '-141857643.23', '-169679753.39', '-155164.80', '-229768.68', '-628137.79', '-84787661.27']
Max y_pred: ['365882.36', '198375982.66', '48158.70', '8872126.22', '88878167.35', '179037296.18', '176223.31', '93979.31', '405067.18', '75813945.78']
Min M: ['-75493264.84', '-99363244.33', '-65570516.48', '-101788670.46', '-93699973.54', '-89446142.97', '-42316856.97', '-146611217.96', '-110637499.56', '-84362999.30']
Max M: ['75493267.83', '99363246.29', '65570520.40', '101788672.98', '93699961.48', '89446146.47', '42316857.73', '146611218.20', '110637500.66', '84362989.00']
Min NTK_labeled: ['-2.54', '-2.58', '-2.22', '-2.26', '-1.88', '-3.52', '-1.37', '-2.13', '-1.89', '-1.94']
Max NTK_labeled: ['71.55', '62.80', '63.94', '60.22', '79.16', '70.08', '44.96', '55.18', '61.07', '68.64']
Min NTK_unlabeled: ['-1.99', '-1.24', '-1.25'

In [25]:
data_params["learning_setting"] = "inductive"
data_params["n_trn_labeled"] = 400
model_params["regularizer"] = 1
n_seeds = 10
run_exp(n_seeds, data_params, model_params, attack_params, verbosity_params, other_params)

Accuracy: ['0.88', '0.88', '0.84', '0.86', '0.87', '0.88', '0.88', '0.87', '0.88', '0.90']
Min y_pred: ['-21.17', '-35.34', '-35.85', '-29.83', '-23.16', '-30.05', '-19.88', '-20.58', '-30.13', '-22.48']
Max y_pred: ['23.62', '41.48', '16.84', '47.61', '19.82', '30.46', '40.67', '17.82', '21.06', '22.07']
Min M: ['-2.06', '-1.94', '-1.56', '-1.91', '-1.90', '-2.28', '-1.89', '-1.56', '-2.47', '-1.52']
Max M: ['3.24', '2.42', '2.70', '2.74', '2.87', '2.40', '2.40', '3.40', '2.57', '3.04']
Min NTK_labeled: ['-2.54', '-2.58', '-2.22', '-2.26', '-1.88', '-3.52', '-1.37', '-2.13', '-1.89', '-1.94']
Max NTK_labeled: ['72.55', '63.80', '64.94', '61.22', '80.16', '71.08', '45.96', '56.18', '62.07', '69.64']
Min NTK_unlabeled: ['-1.99', '-1.24', '-1.25', '-1.03', '-0.93', '-1.45', '-1.57', '-1.44', '-2.17', '-1.18']
Max NTK_unlabeled: ['19.41', '23.67', '20.12', '12.79', '22.19', '24.25', '16.31', '17.83', '21.95', '21.79']
Condition: ['1192', '1165', '1245', '1202', '1191', '1190', '1115', '12

In [26]:
data_params["learning_setting"] = "inductive"
data_params["n_trn_labeled"] = 400
model_params["regularizer"] = 10
n_seeds = 10
run_exp(n_seeds, data_params, model_params, attack_params, verbosity_params, other_params)

Accuracy: ['0.85', '0.89', '0.88', '0.86', '0.88', '0.90', '0.89', '0.87', '0.88', '0.89']
Min y_pred: ['-5.20', '-6.79', '-8.36', '-6.69', '-6.09', '-6.44', '-4.15', '-4.36', '-6.43', '-5.30']
Max y_pred: ['5.01', '7.85', '3.35', '10.11', '5.96', '6.73', '8.08', '3.82', '4.98', '4.55']
Min M: ['-0.34', '-0.33', '-0.41', '-0.33', '-0.44', '-0.44', '-0.31', '-0.30', '-0.40', '-0.33']
Max M: ['0.53', '0.37', '0.39', '0.43', '0.45', '0.40', '0.35', '0.53', '0.40', '0.41']
Min NTK_labeled: ['-2.54', '-2.58', '-2.22', '-2.26', '-1.88', '-3.52', '-1.37', '-2.13', '-1.89', '-1.94']
Max NTK_labeled: ['81.55', '72.80', '73.94', '70.22', '89.16', '80.08', '54.96', '65.18', '71.07', '78.64']
Min NTK_unlabeled: ['-1.99', '-1.24', '-1.25', '-1.03', '-0.93', '-1.45', '-1.57', '-1.44', '-2.17', '-1.18']
Max NTK_unlabeled: ['19.41', '23.67', '20.12', '12.79', '22.19', '24.25', '16.31', '17.83', '21.95', '21.79']
Condition: ['120', '117', '125', '121', '120', '120', '112', '121', '124', '124']


In [27]:
data_params["learning_setting"] = "inductive"
data_params["n_trn_labeled"] = 600
model_params["regularizer"] = 10
n_seeds = 10
run_exp(n_seeds, data_params, model_params, attack_params, verbosity_params, other_params)

Accuracy: ['0.84', '0.87', '0.88', '0.90', '0.89', '0.87', '0.89', '0.81', '0.90', '0.87']
Min y_pred: ['-3.69', '-5.40', '-10.79', '-4.52', '-7.69', '-5.40', '-5.24', '-5.84', '-4.94', '-5.86']
Max y_pred: ['3.48', '7.46', '5.69', '8.77', '6.91', '5.67', '8.66', '4.31', '4.37', '7.79']
Min M: ['-0.38', '-0.34', '-0.43', '-0.40', '-0.37', '-0.46', '-0.35', '-0.43', '-0.32', '-0.52']
Max M: ['0.40', '0.86', '0.43', '0.75', '0.36', '0.47', '0.46', '0.59', '0.45', '0.37']
Min NTK_labeled: ['-2.33', '-2.18', '-2.36', '-1.55', '-2.12', '-2.03', '-2.79', '-1.78', '-2.06', '-2.21']
Max NTK_labeled: ['75.91', '98.01', '93.26', '84.37', '95.18', '84.19', '88.51', '88.99', '80.35', '80.51']
Min NTK_unlabeled: ['-2.20', '-1.44', '-1.22', '-1.67', '-1.97', '-0.74', '-1.43', '-1.56', '-0.87', '-0.94']
Max NTK_unlabeled: ['26.98', '24.61', '31.31', '24.82', '22.27', '19.04', '20.63', '19.59', '19.51', '15.99']
Condition: ['175', '171', '174', '167', '172', '178', '167', '173', '164', '164']


In [28]:
data_params["learning_setting"] = "inductive"
data_params["n_trn_labeled"] = 600
model_params["regularizer"] = 1
n_seeds = 10
run_exp(n_seeds, data_params, model_params, attack_params, verbosity_params, other_params)

Accuracy: ['0.85', '0.85', '0.88', '0.90', '0.90', '0.84', '0.90', '0.84', '0.88', '0.85']
Min y_pred: ['-16.39', '-19.15', '-46.83', '-20.38', '-30.73', '-24.61', '-24.70', '-27.68', '-23.36', '-26.91']
Max y_pred: ['16.31', '33.49', '24.81', '35.88', '30.23', '23.11', '33.83', '23.28', '21.78', '32.14']
Min M: ['-2.37', '-1.59', '-2.98', '-1.97', '-1.58', '-1.58', '-2.25', '-2.56', '-1.61', '-2.41']
Max M: ['2.74', '5.79', '3.02', '3.74', '2.23', '2.75', '3.11', '3.28', '2.87', '2.62']
Min NTK_labeled: ['-2.33', '-2.18', '-2.36', '-1.55', '-2.12', '-2.03', '-2.79', '-1.78', '-2.06', '-2.21']
Max NTK_labeled: ['66.91', '89.01', '84.26', '75.37', '86.18', '75.19', '79.51', '79.99', '71.35', '71.51']
Min NTK_unlabeled: ['-2.20', '-1.44', '-1.22', '-1.67', '-1.97', '-0.74', '-1.43', '-1.56', '-0.87', '-0.94']
Max NTK_unlabeled: ['26.98', '24.61', '31.31', '24.82', '22.27', '19.04', '20.63', '19.59', '19.51', '15.99']
Condition: ['1739', '1704', '1735', '1659', '1711', '1776', '1664', '17

## Experiments for NTK matrices sizes

In [10]:
data_params["learning_setting"] = "transductive"
data_params["n_trn_labeled"] = 400
model_params["regularizer"] = 1e-8
n_seeds = 10
run_exp(n_seeds, data_params, model_params, attack_params, verbosity_params, other_params)

Accuracy: ['0.80', '0.79', '0.83', '0.80', '0.72', '0.81', '0.78', '0.73', '0.83', '0.77']
Min y_pred: ['-3.10', '-6.50', '-5.07', '-4.63', '-5.75', '-9.19', '-4.92', '-8.77', '-4.17', '-6.36']
Max y_pred: ['4.75', '7.57', '4.65', '4.18', '5.16', '4.53', '2.66', '3.69', '5.27', '4.42']
Min M: ['-1.28', '-1.50', '-1.52', '-1.73', '-1.49', '-1.48', '-1.72', '-1.64', '-1.82', '-1.48']
Max M: ['3.19', '3.06', '3.14', '2.53', '3.71', '3.05', '2.41', '2.74', '3.00', '3.24']
Min NTK_labeled: ['-0.74', '-1.50', '-0.93', '-1.32', '-1.48', '-1.29', '-0.93', '-1.34', '-1.81', '-1.84']
Max NTK_labeled: ['71.55', '62.80', '62.61', '60.22', '68.57', '70.08', '44.96', '55.18', '61.07', '68.64']
Min NTK_unlabeled: ['-1.99', '-1.24', '-1.25', '-1.03', '-0.93', '-1.45', '-1.57', '-1.44', '-2.17', '-1.18']
Max NTK_unlabeled: ['19.41', '23.67', '20.12', '12.79', '22.19', '24.25', '16.31', '17.83', '21.95', '21.79']
Condition: ['387206', '89317051565', '393982', '91860584860', '386794', '394657', '361407',

In [19]:
data_params["learning_setting"] = "transductive"
data_params["n_trn_labeled"] = 400
model_params["regularizer"] = 1e-3
n_seeds = 10
run_exp(n_seeds, data_params, model_params, attack_params, verbosity_params, other_params)

Accuracy: ['0.82', '0.81', '0.82', '0.81', '0.76', '0.83', '0.80', '0.77', '0.84', '0.79']
Min y_pred: ['-2.79', '-5.96', '-4.43', '-4.28', '-6.00', '-9.09', '-4.50', '-7.84', '-3.73', '-5.93']
Max y_pred: ['4.46', '6.86', '4.43', '4.31', '4.75', '4.26', '2.95', '3.65', '4.81', '4.02']
Min M: ['-1.20', '-1.40', '-1.37', '-1.54', '-1.36', '-1.36', '-1.59', '-1.57', '-1.65', '-1.39']
Max M: ['3.10', '2.91', '3.05', '2.44', '3.49', '2.88', '2.36', '2.63', '2.95', '3.12']
Min NTK_labeled: ['-0.74', '-1.50', '-0.93', '-1.32', '-1.48', '-1.29', '-0.93', '-1.34', '-1.81', '-1.84']
Max NTK_labeled: ['71.55', '62.80', '62.61', '60.22', '68.58', '70.08', '44.96', '55.19', '61.07', '68.65']
Min NTK_unlabeled: ['-1.99', '-1.24', '-1.25', '-1.03', '-0.93', '-1.45', '-1.57', '-1.44', '-2.17', '-1.18']
Max NTK_unlabeled: ['19.41', '23.67', '20.12', '12.79', '22.19', '24.25', '16.31', '17.83', '21.95', '21.79']
Condition: ['272972', '893171', '274119', '918607', '272943', '275383', '255265', '287298',

In [20]:
data_params["learning_setting"] = "transductive"
data_params["n_trn_labeled"] = 400
model_params["regularizer"] = 1e-2
n_seeds = 10
run_exp(n_seeds, data_params, model_params, attack_params, verbosity_params, other_params)

Accuracy: ['0.87', '0.87', '0.84', '0.84', '0.83', '0.86', '0.87', '0.82', '0.88', '0.82']
Min y_pred: ['-1.98', '-4.33', '-2.63', '-2.94', '-6.02', '-8.36', '-3.73', '-4.67', '-3.57', '-4.67']
Max y_pred: ['3.51', '4.80', '3.68', '4.68', '3.42', '3.28', '4.20', '3.77', '3.45', '2.82']
Min M: ['-0.85', '-0.99', '-0.87', '-0.98', '-0.87', '-0.87', '-1.07', '-1.21', '-1.01', '-1.00']
Max M: ['2.57', '2.40', '2.51', '2.03', '2.55', '2.09', '2.09', '2.27', '2.67', '2.55']
Min NTK_labeled: ['-0.74', '-1.50', '-0.93', '-1.32', '-1.48', '-1.29', '-0.93', '-1.34', '-1.81', '-1.84']
Max NTK_labeled: ['71.56', '62.81', '62.62', '60.23', '68.58', '70.09', '44.97', '55.19', '61.08', '68.65']
Min NTK_unlabeled: ['-1.99', '-1.24', '-1.25', '-1.03', '-0.93', '-1.45', '-1.57', '-1.44', '-2.17', '-1.18']
Max NTK_unlabeled: ['19.41', '23.67', '20.12', '12.79', '22.19', '24.25', '16.31', '17.83', '21.95', '21.79']
Condition: ['74681', '89318', '73331', '91862', '74797', '74028', '70066', '76129', '95991'

In [21]:
data_params["learning_setting"] = "transductive"
data_params["n_trn_labeled"] = 400
model_params["regularizer"] = 1e-1
n_seeds = 10
run_exp(n_seeds, data_params, model_params, attack_params, verbosity_params, other_params)

Accuracy: ['0.92', '0.90', '0.86', '0.88', '0.85', '0.92', '0.91', '0.84', '0.91', '0.88']
Min y_pred: ['-1.84', '-2.82', '-2.88', '-1.78', '-5.19', '-7.24', '-2.81', '-2.18', '-3.28', '-2.46']
Max y_pred: ['2.58', '3.24', '2.41', '4.13', '2.95', '2.28', '4.58', '4.04', '2.37', '1.69']
Min M: ['-0.37', '-0.36', '-0.33', '-0.36', '-0.35', '-0.42', '-0.47', '-0.47', '-0.45', '-0.42']
Max M: ['1.49', '1.39', '1.28', '1.32', '1.41', '1.35', '1.32', '1.51', '1.79', '1.47']
Min NTK_labeled: ['-0.74', '-1.50', '-0.93', '-1.32', '-1.48', '-1.29', '-0.93', '-1.34', '-1.81', '-1.84']
Max NTK_labeled: ['71.65', '62.90', '62.71', '60.32', '68.67', '70.18', '45.06', '55.28', '61.17', '68.74']
Min NTK_unlabeled: ['-1.99', '-1.24', '-1.25', '-1.03', '-0.93', '-1.45', '-1.57', '-1.44', '-2.17', '-1.18']
Max NTK_unlabeled: ['19.41', '23.67', '20.12', '12.79', '22.19', '24.25', '16.31', '17.83', '21.95', '21.79']
Condition: ['9038', '8933', '8810', '9187', '9057', '8907', '8488', '9118', '9600', '9116']

In [22]:
data_params["learning_setting"] = "transductive"
data_params["n_trn_labeled"] = 400
model_params["regularizer"] = 1
n_seeds = 10
run_exp(n_seeds, data_params, model_params, attack_params, verbosity_params, other_params)

Accuracy: ['0.88', '0.89', '0.88', '0.89', '0.85', '0.93', '0.90', '0.88', '0.91', '0.90']
Min y_pred: ['-1.69', '-2.19', '-3.34', '-1.69', '-4.31', '-5.69', '-1.69', '-1.79', '-2.69', '-2.20']
Max y_pred: ['2.09', '2.93', '1.68', '3.38', '2.99', '1.87', '4.01', '2.85', '2.08', '1.64']
Min M: ['-0.14', '-0.17', '-0.11', '-0.16', '-0.18', '-0.15', '-0.13', '-0.12', '-0.18', '-0.11']
Max M: ['0.87', '0.88', '0.65', '0.68', '0.77', '0.65', '0.58', '0.81', '0.70', '0.79']
Min NTK_labeled: ['-0.74', '-1.50', '-0.93', '-1.32', '-1.48', '-1.29', '-0.93', '-1.34', '-1.81', '-1.84']
Max NTK_labeled: ['72.55', '63.80', '63.61', '61.22', '69.57', '71.08', '45.96', '56.18', '62.07', '69.64']
Min NTK_unlabeled: ['-1.99', '-1.24', '-1.25', '-1.03', '-0.93', '-1.45', '-1.57', '-1.44', '-2.17', '-1.18']
Max NTK_unlabeled: ['19.41', '23.67', '20.12', '12.79', '22.19', '24.25', '16.31', '17.83', '21.95', '21.79']
Condition: ['924', '894', '900', '920', '926', '910', '868', '931', '961', '931']


In [112]:
data_params["learning_setting"] = "transductive"
data_params["n_trn_labeled"] = 400
acc_l = []
min_ypred = []
max_ypred = []
cond = []
min_M = []
max_M = []
min_ntklabeled = []
max_ntklabeled = []
min_ntkunlabeled = []
max_ntkunlabeled = []
for seed in range(10):
    data_params["seed"] = seed
    res = run(data_params, model_params, attack_params, verbosity_params, other_params, seed)
    acc_l.append(res["accuracy_l"][0])
    min_ypred.append(res["min_ypred"][0])
    max_ypred.append(res["max_ypred"][0])
    min_M.append(res["min_M"][0])
    max_M.append(res["max_M"][0])
    min_ntklabeled.append(res["min_ntklabeled"][0])
    max_ntklabeled.append(res["max_ntklabeled"][0])
    min_ntkunlabeled.append(res["min_ntkunlabeled"][0])
    max_ntkunlabeled.append(res["max_ntkunlabeled"][0])
    cond.append(res["cond"])
print(f"Accuracy: {get_str_l(acc_l)}")
print(f"Min y_pred: {get_str_l(min_ypred)}")
print(f"Max y_pred: {get_str_l(max_ypred)}")
print(f"Min M: {get_str_l(min_M)}")
print(f"Max M: {get_str_l(max_M)}")
print(f"Min NTK_labeled: {get_str_l(min_ntklabeled)}")
print(f"Max NTK_labeled: {get_str_l(max_ntklabeled)}")
print(f"Min NTK_unlabeled: {get_str_l(min_ntkunlabeled)}")
print(f"Max NTK_unlabeled: {get_str_l(max_ntkunlabeled)}")
print(f"Condition: {get_str_l(cond, precision=0)}")

0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
Accuracy: ['0.80', '0.79', '0.83', '0.79', '0.74', '0.82', '0.78', '0.75', '0.77', '0.78']
Min y_pred: ['-3.09', '-6.62', '-5.19', '-4.70', '-5.72', '-9.12', '-4.91', '-8.62', '-5.72', '-6.30']
Max y_pred: ['4.57', '7.56', '4.66', '4.09', '5.15', '4.81', '2.68', '3.77', '4.67', '4.54']
Min M: ['-1.28', '-2.77', '-1.51', '-35.60', '-1.49', '-1.49', '-1.71', '-1.64', '-130.30', '-1.48']
Max M: ['3.20', '3.05', '3.14', '35.61', '3.71', '3.05', '2.41', '2.74', '130.27', '3.24']
Min NTK_labeled: ['-0.74', '-1.50', '-0.93', '-1.32', '-1.48', '-1.29', '-0.93', '-1.34', '-1.81', '-1.84']
Max NTK_label

In [113]:
data_params["learning_setting"] = "inductive"
data_params["n_trn_labeled"] = 400
acc_l = []
min_ypred = []
max_ypred = []
cond = []
min_M = []
max_M = []
min_ntklabeled = []
max_ntklabeled = []
min_ntkunlabeled = []
max_ntkunlabeled = []
for seed in range(10):
    data_params["seed"] = seed
    res = run(data_params, model_params, attack_params, verbosity_params, other_params, seed)
    acc_l.append(res["accuracy_l"][0])
    min_ypred.append(res["min_ypred"][0])
    max_ypred.append(res["max_ypred"][0])
    min_M.append(res["min_M"][0])
    max_M.append(res["max_M"][0])
    min_ntklabeled.append(res["min_ntklabeled"][0])
    max_ntklabeled.append(res["max_ntklabeled"][0])
    min_ntkunlabeled.append(res["min_ntkunlabeled"][0])
    max_ntkunlabeled.append(res["max_ntkunlabeled"][0])
    cond.append(res["cond"])
print(f"Accuracy: {get_str_l(acc_l)}")
print(f"Min y_pred: {get_str_l(min_ypred)}")
print(f"Max y_pred: {get_str_l(max_ypred)}")
print(f"Min M: {get_str_l(min_M)}")
print(f"Max M: {get_str_l(max_M)}")
print(f"Min NTK_labeled: {get_str_l(min_ntklabeled)}")
print(f"Max NTK_labeled: {get_str_l(max_ntklabeled)}")
print(f"Min NTK_unlabeled: {get_str_l(min_ntkunlabeled)}")
print(f"Max NTK_unlabeled: {get_str_l(max_ntkunlabeled)}")
print(f"Condition: {get_str_l(cond, precision=0)}")

0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
Accuracy: ['0.54', '0.33', '0.76', '0.38', '0.49', '0.56', '0.57', '0.32', '0.68', '0.56']
Min y_pred: ['-97490.87', '-48360.84', '-50460.89', '-28497.31', '-142206.29', '-19802.02', '-28167.63', '-40436.56', '-75658.87', '-40327.67']
Max y_pred: ['182670.60', '54618.05', '55201.70', '10045.66', '115411.53', '7496.70', '32783.82', '47047.44', '23566.33', '10303.28']
Min M: ['-35904.57', '-20096.42', '-19311.67', '-23325.17', '-64763.02', '-13258.93', '-42183.66', '-20909.61', '-108140.02', '-161703.38']
Max M: ['61231.26', '20089.39', '24244.20', '23340.64', '64742.14', '20135.12', '42191.24',

## Experiments for y_pred size

In [71]:
data_params["learning_setting"] = "transductive"
data_params["n_trn_labeled"] = 400
acc_l = []
min_ypred = []
max_ypred = []
cond = []
for seed in range(10):
    data_params["seed"] = seed
    res = run(data_params, model_params, attack_params, verbosity_params, other_params, seed)
    acc_l.append(res["accuracy_l"][0])
    min_ypred.append(res["min_ypred"][0])
    max_ypred.append(res["max_ypred"][0])
    cond.append(res["cond"])
print(acc_l)
print(min_ypred)
print(max_ypred)
print(cond)

[0.7999999523162842, 0.7949999570846558, 0.8299999833106995, 0.7949999570846558, 0.7400000095367432, 0.824999988079071, 0.7799999713897705, 0.75, 0.7749999761581421, 0.7799999713897705]
[-3.0920467578784803, -6.6160343128658, -5.194746775288973, -4.699020576493933, -5.718078334268294, -9.115392589081491, -4.905361471048922, -8.61613686394207, -5.719317177977954, -6.302207013707667]
[4.570836528482572, 7.558606664491745, 4.6590389998099635, 4.0878273364676465, 5.15241891023212, 4.80595657486644, 2.6751935812229077, 3.7679468135510303, 4.6712427287199105, 4.540205492559701]
[389053.63117462175, 17227532.286783576, 386959.6250253748, 175422543.52912518, 382813.985553917, 397663.6694353313, 367239.72382285836, 412507.3546088346, 678918974.0532519, 413583.01504568243]


In [102]:
data_params["learning_setting"] = "inductive"
p_unnorm = data_params["avg_within_class_degree"]
data_params["avg_within_class_degree"] = 1.58 * 2 * 100
q_unnorm = data_params["avg_between_class_degree"]
data_params["avg_between_class_degree"] = 0.37 * 2 * 70
acc_l = []
min_ypred = []
max_ypred = []
cond = []
for seed in range(10):
    data_params["seed"] = seed
    res = run(data_params, model_params, attack_params, verbosity_params, other_params, seed)
    acc_l.append(res["accuracy_l"][0])
    min_ypred.append(res["min_ypred"][0])
    max_ypred.append(res["max_ypred"][0])
    cond.append(res["cond"])
print(acc_l)
print(min_ypred)
print(max_ypred)
print(cond)

0.6326326326326326
0.1037037037037037
0.6326326326326326
0.1037037037037037
0.6326326326326326
0.1037037037037037
0.6326326326326326
0.1037037037037037
0.6326326326326326
0.1037037037037037
0.6326326326326326
0.1037037037037037
0.6326326326326326
0.1037037037037037
0.6326326326326326
0.1037037037037037
0.6326326326326326
0.1037037037037037
0.6326326326326326
0.1037037037037037
[1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0]
[-198887.16105522725, -570682.9825172557, -1447257.8573979978, -37860.66165276896, -55926.801587639144, -290040.57506164117, -350577.9186986564, -391894.0606868819, -590375.6752174892, -351662.7199801542]
[320796.28391054017, 394927.00222813245, 1027750.7295307582, 28479.814787887735, 138296.76934363693, 467328.4684214182, 303189.61444266746, 288720.0622061163, 543805.0602562819, 1210417.7903377097]
[218709333701.72552, 395523266595.4415, 4047205108377.5347, 367468484718.35284, 328162745884.328, 297005777329.7517, 252244108489.32474, 298833724882.89355, 111525099

In [ ]:
data_params["learning_setting"] = "transductive"
acc_l = []
min_ypred = []
max_ypred = []
cond = []
for seed in range(10):
    data_params["seed"] = seed
    res = run(data_params, model_params, attack_params, verbosity_params, other_params, seed)
    acc_l.append(res["accuracy_l"][0])
    min_ypred.append(res["min_ypred"][0])
    max_ypred.append(res["max_ypred"][0])
    cond.append(res["cond"])
print(acc_l)
print(min_ypred)
print(max_ypred)
print(cond)

-> clear correlation low y_pred values mean good predictions (|min/max| < 10)

In [68]:
data_params["learning_setting"] = "inductive"
acc_l = []
min_ypred = []
max_ypred = []
cond = []
for seed in range(10):
    data_params["seed"] = seed
    res = run(data_params, model_params, attack_params, verbosity_params, other_params, seed)
    acc_l.append(res["accuracy_l"][0])
    min_ypred.append(res["min_ypred"][0])
    max_ypred.append(res["max_ypred"][0])
    cond.append(res["cond"])
print(acc_l)
print(min_ypred)
print(max_ypred)
print(cond)

[0.6899999976158142, 0.5550000071525574, 0.5799999833106995, 0.4449999928474426, 0.3349999785423279, 0.675000011920929, 0.44999998807907104, 0.44999998807907104, 0.6100000143051147, 0.5]
[-30247.846224752207, -95078.36257414054, -26783.68457652904, -21873.26632158264, -48363.07600911075, -28243.6273096534, -16259.459278168943, -33341.27322340436, -24276.7436774647, -45002.83794352863]
[120458.61990854447, 46644.489896458486, 96389.28003963197, 15950.742201795094, 16671.90547960927, 16726.042436049684, 56953.97201926261, 32503.00367441383, 19232.59269619555, 50973.9443629398]
[481320232.12236416, 904617224.7448723, 390462481.61452425, 108613454.24243383, 443483057.5212938, 171410416.884777, 640581624.5041744, 218038721.29718912, 216075456.717386, 216347577.68922815]


In [75]:
data_params["learning_setting"] = "transductive"
data_params["n_trn_labeled"] = 2000
data_params["n_val"] = 500
data_params["n_test"] = 1000
acc_l = []
min_ypred = []
max_ypred = []
cond = []
for seed in range(10):
    print(f"Seed: {seed}")
    data_params["seed"] = seed
    res = run(data_params, model_params, attack_params, verbosity_params, other_params, seed)
    acc_l.append(res["accuracy_l"][0])
    min_ypred.append(res["min_ypred"][0])
    max_ypred.append(res["max_ypred"][0])
    cond.append(res["cond"])
print(acc_l)
print(min_ypred)
print(max_ypred)
print(cond)

Seed: 0
Seed: 1
Seed: 2
Seed: 3
Seed: 4
Seed: 5
Seed: 6
Seed: 7
Seed: 8
Seed: 9
[0.5210000276565552, 0.7950000166893005, 0.7790000438690186, 0.7770000100135803, 0.8180000185966492, 0.7910000085830688, 0.7950000166893005, 0.7890000343322754, 0.5380000472068787, 0.8110000491142273]
[-296.59076156107926, -4.52564114908652, -5.673511484411298, -8.380375245901904, -5.813458484371335, -11.09270211128861, -5.931646827522416, -5.443586191531857, -69.78675415576794, -5.146435286173144]
[240.74777676288156, 8.062967090310071, 10.359658009010246, 7.045663003687437, 5.7407786667211695, 5.688827342343328, 5.933379658153002, 5.643244483079846, 57.0348302470963, 5.971274784526761]
[20291603317.12647, 618988173.5184208, 481220517.795961, 379796531.44778395, 407686915.44799507, 849027.1857061002, 853581.9783354799, 992819998.149226, 1871702270.9986954, 817203.3807346252]


In [40]:
# Transductive
data_params["learning_setting"] = "transductive"
acc_l = []
for seed in range(30):
    data_params["seed"] = seed
    res = run(data_params, model_params, attack_params, verbosity_params, other_params, seed)
    acc_l.append(res["accuracy_l"][0])
print(acc_l)
print(np.mean(acc_l))
print(np.std(acc_l))

2023-11-28 11:09:06 (INFO): Starting experiment exp_ntk with configuration:
2023-11-28 11:09:06 (INFO): data_params: {'learning_setting': 'transductive', 'classes': 2, 'n_trn_labeled': 600, 'n_trn_unlabeled': 0, 'n_val': 200, 'n_test': 200, 'sigma': 1, 'avg_within_class_degree': 3.16, 'avg_between_class_degree': 0.74, 'K': 1.5, 'seed': 0}
2023-11-28 11:09:06 (INFO): model_params: {'label': 'GCN', 'model': 'GCN', 'normalization': 'row_normalization', 'depth': 1}
2023-11-28 11:09:06 (INFO): attack_params: {'attack': 'random', 'epsilon_list': [0], 'attack_setting': 'evasion'}
2023-11-28 11:09:06 (INFO): verbosity_params: {'debug_lvl': 'warning'}
2023-11-28 11:09:06 (INFO): other_params: {'device': '0', 'dtype': torch.float64, 'allow_tf32': False}


2023-11-28 11:09:06 (INFO): seed: 0
2023-11-28 11:09:06 (INFO): Currently on gpu device cuda:0
2023-11-28 11:09:06 (INFO): Accuracy 0.6949999928474426 for epsilon 0
2023-11-28 11:09:06 (INFO): Starting experiment exp_ntk with configuration:
2023-11-28 11:09:06 (INFO): data_params: {'learning_setting': 'transductive', 'classes': 2, 'n_trn_labeled': 600, 'n_trn_unlabeled': 0, 'n_val': 200, 'n_test': 200, 'sigma': 1, 'avg_within_class_degree': 3.16, 'avg_between_class_degree': 0.74, 'K': 1.5, 'seed': 1}
2023-11-28 11:09:06 (INFO): model_params: {'label': 'GCN', 'model': 'GCN', 'normalization': 'row_normalization', 'depth': 1}
2023-11-28 11:09:06 (INFO): attack_params: {'attack': 'random', 'epsilon_list': [0], 'attack_setting': 'evasion'}
2023-11-28 11:09:06 (INFO): verbosity_params: {'debug_lvl': 'warning'}
2023-11-28 11:09:06 (INFO): other_params: {'device': '0', 'dtype': torch.float64, 'allow_tf32': False}
2023-11-28 11:09:06 (INFO): seed: 1
2023-11-28 11:09:06 (INFO): Currently on gpu 

[0.6949999928474426, 0.7899999618530273, 0.7599999904632568, 0.8149999976158142, 0.7649999856948853, 0.7849999666213989, 0.7899999618530273, 0.4899999797344208, 0.7899999618530273, 0.5049999952316284, 0.8050000071525574, 0.7199999690055847, 0.7299999594688416, 0.73499995470047, 0.7999999523162842, 0.699999988079071, 0.7749999761581421, 0.7400000095367432, 0.7249999642372131, 0.7699999809265137, 0.8050000071525574, 0.7249999642372131, 0.8050000071525574, 0.7899999618530273, 0.5049999952316284, 0.7400000095367432, 0.6800000071525574, 0.7099999785423279, 0.7549999952316284, 0.7199999690055847]
0.7306666483481725
0.08507578059838189


In [13]:
import numpy as np

In [14]:
a = np.arange(10)
b = np.arange(5)
np.concatenate((a, b))

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4])